In [ ]:
!pip install dist/alquimia-fair-forge-0.0.1.tar.gz -q

In [2]:
from helpers.retriever import LocalRetriever
from fair_forge.metrics import Bias
from pydantic import SecretStr
import os

In [13]:
from getpass import getpass
guardian_api_key = SecretStr(getpass("Please enter your Guardian LLM API key: "))

In [14]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
bias_index = f"{dataset}-bias"
guardian_temperature = 0.5
max_tokens = 5

In [15]:
## GRANITE
GUARDIAN_URL = os.environ.get("GUARDIAN_URL","https://runtime.apps.hostmydemo.online")
GUARDIAN_MODEL_NAME = os.environ.get("GUARDIAN_MODEL_NAME","ibm-granite/granite-guardian-3.1-2b")
GUARDIAN_API_KEY = guardian_api_key

In [5]:
## LLAMA GUARD
GUARDIAN_URL = "https://api.groq.com/openai"
GUARDIAN_MODEL_NAME = "meta-llama/Llama-Guard-4-12B"
hf_token = SecretStr(getpass("Please enter your Hugging Face token: "))
os.environ["HF_TOKEN"] = hf_token.get_secret_value()
GUARDIAN_API_KEY = guardian_api_key

In [16]:
from fair_forge.guardians import LLamaGuard,IBMGranite
from fair_forge.guardians.llms.providers import OpenAIGuardianProvider
from fair_forge.schemas import GuardianLLMConfig

In [ ]:
metrics= Bias.run(
    LocalRetriever,
    guardian = IBMGranite,
    confidence_level= 0.90,
    config = GuardianLLMConfig(
        model= GUARDIAN_MODEL_NAME,
        api_key= GUARDIAN_API_KEY.get_secret_value(),
        url=GUARDIAN_URL,
        temperature=guardian_temperature,
        provider=OpenAIGuardianProvider,
        logprobs= False
    ),
    verbose=True
)

In [ ]:
for metric in metrics:
    print("---")
    print(f"Protected attribute: {metric.guardian_attributes_confidence_interval.protected_attribute}")
    print(f"Confidence level: {metric.guardian_attributes_confidence_interval.confidence_level}")
    print(f"Lower bound: {metric.guardian_attributes_confidence_interval.lower_bound}")
    print(f"Probability: {metric.guardian_attributes_confidence_interval.probability}")
    print(f"Upper bound: {metric.guardian_attributes_confidence_interval.upper_bound}")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
def flatten_guard_metrics(metrics):
    flattened = []
    for metric in metrics:
        for risk in metric.risks:
            flattened.append(
                {
                    "session_id": metric.session_id,
                    "qa_id": metric.qa_id,
                    "assistant_id": metric.assistant_id,
                    'bias_guard_is_risk': risk.is_risk,
                    'bias_guard_type': risk.risk_type,
                    'bias_guard_probability': risk.probability
                }
            )
    return flattened
flattened = flatten_guard_metrics(metrics)

In [ ]:
mapping_bias = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "bias_guard_is_risk": {"type": "boolean"},
      "bias_guard_type": {"type": "text"},
      "bias_guard_probability": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}

In [ ]:
recreate_index(bias_index, mapping_bias)

In [ ]:
docs = []
for flatten in flattened:
    docs.append({
            "_index": bias_index,
            "_source": flatten
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")